<a href="https://colab.research.google.com/github/65-1157/65-1157/blob/main/Dynamic-Programming-Algorithms-Codes%20/2025_10_27_28_Viterbi_detalhes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
from typing import List, Tuple, Dict

## 2) Algoritmo Viterbi (POS Tagging)

**Objetivo:** Encontrar a sequência de tags (estados ocultos) $\mathbf{S} = s_1, s_2, \dots, s_T$ que maximiza a probabilidade de observarmos a frase $\mathbf{O} = o_1, o_2, \dots, o_T$.

**Definição de DP:**
O Viterbi utiliza a Programação Dinâmica para calcular: $\mathbf{V}_{k}(\text{tag}) = \text{log-prob da sequência mais provável até o tempo } k, \text{terminando em tag}$.

**Por que Log-Probabilidades?**

| Característica | Propósito |
| :--- | :--- |
| **Robustez Numérica** | Em vez de multiplicar probabilidades pequenas ($P_1 \cdot P_2 \cdot P_3$), somamos seus logaritmos ($\log(P_1) + \log(P_2) + \log(P_3)$). |
| **Prevenção de Underflow** | Evita que probabilidades muito pequenas sejam arredondadas para zero pelo computador, garantindo a precisão do cálculo. |

**Passo Dinâmico (Fórmula em Log):**

$\mathbf{V}_{k}(t) = \max_{\text{ant}} \left[ \mathbf{V}_{k-1}(\text{ant}) + \log(\mathbf{P}_{\text{trans}} (\text{ant} \to t)) + \log(\mathbf{P}_{\text{emiss}} (t \to o_k)) \right]$

* **Complexidade:** $\mathcal{O}(T \cdot |\text{Tags}|^2)$, onde $T$ é o tamanho da frase.

In [ ]:
UNKNOWN_WORD_PROB = 1e-6 # pequeno número para evitar divisao por zero
LOG_INF = -float('inf') # valor muito negativo, para operar com log(0)

def get_log_prob(prob: float) -> float:
    """ Transform probabilidade em log-prob."""
    # Retorna log(prob) se for positivo, senão -inf (log(0))
    return math.log(prob) if prob > 0 else LOG_INF

# -----------------------------------------------------------------
# Atribuir probabilidades para Atividades x Clima Local
# Tags: SOL (Estado Oculto), CHUVA (Estado Oculto)
# -----------------------------------------------------------------
tags = ["SOL", "CHUVA"] # variaveis de clima

# Probabilidade inicial
p_inicio = {"SOL": 0.8, "CHUVA": 0.2} # levamentamento anterior

# Probabilidade de um dia para outro - condicional. Soma das probabilidades = 1
p_trans = {
    "SOL": {"SOL": 0.7, "CHUVA": 0.3},
    "CHUVA": {"SOL": 0.4, "CHUVA": 0.6},
}

# Probabilidade de atividades em funcao do clima local
# Nao ha obrigacao de probabilidade somar 1.0.
p_emiss = {
    "SOL": {"bicicleta": 0.6, "praia": 0.3, "guarda-chuva": 0.05, "trabalhar": 0.2},
    "CHUVA": {"bicicleta": 0.1, "praia": 0.05, "guarda-chuva": 0.5, "trabalhar": 0.3},
}

# ALGORITMO VITERBI
# -----------------------------------------------------------------

def viterbi(palavras: List[str]) -> Tuple[List[str], List[Dict]]:
    """
    Implement Algoritmo Viterbi (DP) para encontrar a sequência
    de tags mais provável (estados ocultos).
    """
    V = [{}]      # Tabela DP: armazena a melhor log-probabilidade até o passo k
    back = [{}]   # Tabela Backpointers: armazena a tag anterior que levou ao máximo
    print(f"Palavras: {palavras}")
    breakpoint()

    # --- 1. Inicialização (k=0) ---
    palavra_0 = palavras[0]
    for t in tags:
        emiss_prob = p_emiss[t].get(palavra_0, UNKNOWN_WORD_PROB)

        # V[0][t] = log(P_inicio[t]) + log(P_emissao[t | palavra_0])
        V[0][t] = round(get_log_prob(p_inicio[t]) + get_log_prob(emiss_prob),3)
        print(f"V[0][{t}] = {V[0][t]}")
        back[0][t] = None
        breakpoint()

    # --- 2. Passos Dinâmicos (k > 0) ---
    for k in range(1, len(palavras)):
        V.append({})
        back.append({})
        palavra_k = palavras[k]
        print(f"\nPalavra {k}: {palavra_k}")
        breakpoint()

        for t in tags: # t = tag atual
            emiss_prob = round(p_emiss[t].get(palavra_k, UNKNOWN_WORD_PROB),3)
            log_emiss = round(get_log_prob(emiss_prob),3)
            print(f"P(E | {t}) = {emiss_prob}")
            print(f"log(P(E | {t})) = {log_emiss}")
            breakpoint()

            melhor_lp = LOG_INF
            melhor_ant = None

            # DP: Itera sobre todas as tags anteriores ('ant')
            for ant in tags:
                log_trans = round(get_log_prob(p_trans[ant][t]),3)
                print(f"P(T | {ant} -> {t}) = {p_trans[ant][t]}")
                print(f"log(P(T | {ant} -> {t})) = {log_trans}")
                breakpoint()

                # V[k][t] = max( V[k-1][ant] + log(P_trans) + log(P_emiss) )
                lp = V[k-1][ant] + log_trans + log_emiss
                print(f"V[{k-1}][{ant}] + log(P(T | {ant} -> {t})) + log(P(E | {t})) = {lp}")
                breakpoint()

                if lp > melhor_lp:
                    melhor_lp, melhor_ant = lp, ant

            V[k][t] = melhor_lp
            back[k][t] = melhor_ant

    # --- 3. Término e Reconstrução do Caminho ---
    T = len(palavras) - 1

    # Encontra a tag final com a maior probabilidade total
    ultimo = max(tags, key=lambda t: V[T][t])
    print(f"Tag final: {ultimo}")
    caminho = [ultimo]
    print(f"Caminho: {caminho}")
    breakpoint()

    # Backtrace: segue os ponteiros 'back'
    for k in range(T, 0, -1):
        ultimo = back[k][ultimo]
        print(f"Caminho: {caminho}")
        caminho.append(ultimo)

    caminho.reverse()
    print(f"Caminho: {caminho}")
    print(f"V: {V}")
    print(f"back: {back}")
    return caminho, V

In [ ]:
# --- EXECUÇÃO DO NOVO EXEMPLO (CLIMA) ---

frase_atividade = "bicicleta trabalhar guarda-chuva"
palavras = frase_atividade.lower().split()

# Executa o algoritmo Viterbi
trajeto, V = viterbi(palavras)

print("-------------------------------------------------------")
print("Análise de Previsão de Tempo Oculto (Viterbi)")
print(f"Atividades Observadas (dias): {palavras}")
print(f"Clima Mais Provável (tags): {trajeto}")
print("-------------------------------------------------------")

# Imprime o detalhe da probabilidade final (opcional, para didática)
log_prob_final = max(V[-1].values())
print(f"Log-Probabilidade do Caminho Ótimo: {log_prob_final:.4f}")

Palavras: ['bicicleta', 'trabalhar', 'guarda-chuva']
> /tmp/ipython-input-3322221107.py(45)viterbi()
     43 
     44     # --- 1. Inicialização (k=0) ---
---> 45     palavra_0 = palavras[0]
     46     for t in tags:
     47         emiss_prob = p_emiss[t].get(palavra_0, UNKNOWN_WORD_PROB)

--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
V[0][SOL] = -0.734
> /tmp/ipython-input-3322221107.py(46)viterbi()
     44     # --- 1. Inicialização (k=0) ---
     45     palavra_0 = palavras[0]
---> 46     for t in tags:
     47         emiss_prob = p_emiss[t].get(palavra_0, UNKNOWN_WORD_PROB)
     48 

--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
V[0][CHUVA] = -3.912
> /tmp/ipython-input-3322221107.py(46)viterbi()
     44     # --- 1. Inicialização (k=0) ---
     45     palavra_0 = palavras[0]
---> 46     for t in tags:
     47         emiss_prob = p_emiss[t].get(palavra_0, UNKNOWN_WORD_PROB)
     48 


Palavra 1: trabalhar
> /tmp/ipython-input-3322221107.py(63)vit